# 关于高德地图天气查询API的调用


In [1]:
import pandas as pd
import requests
from pandas.io.json import json_normalize
key_peng ="96b267374a02ffa474503092d0b3eb4d"# 改为本人获取的key

In [40]:
#封装名为weatherInfo的函数
def weatherInfo(city,extensions='base')->dict:
    url = "https://restapi.amap.com/v3/weather/weatherInfo?parameters"
    params={
        "key":key_peng,
        "city":city,
        "extensions":extensions,
        "output":"json"    
    }
    response = requests.get(url,params=params)
    data = response.json()
    return data
    
长沙_天气_实况 =  weatherInfo(city='长沙市',extensions='base')
长沙_天气_实况

{'status': '1',
 'count': '1',
 'info': 'OK',
 'infocode': '10000',
 'lives': [{'province': '湖南',
   'city': '长沙市',
   'adcode': '430100',
   'weather': '多云',
   'temperature': '35',
   'winddirection': '南',
   'windpower': '4',
   'humidity': '52',
   'reporttime': '2020-07-17 14:24:30'}]}

In [42]:
df_长沙_天气_实况 = pd.json_normalize(长沙_天气_实况['lives'])
df_长沙_天气_实况

,province,city,adcode,weather,temperature,winddirection,windpower,humidity,reporttime
0,湖南,长沙市,430100,多云,35,南,4,52,2020-07-17 14:24:30


# 关于高德地图路径规划API的调用

## 步行路径规划

In [12]:
# 封装名为walking的函数
def walking(origin,destination,sig=None)->dict:
    url = 'https://restapi.amap.com/v3/direction/walking?parameters'
    params={
        'key':key_peng,
        'origin':origin,
        'destination':destination,
        'output':'json'
    }
    response = requests.get(url,params=params)
    data = response.json()
    return data

In [9]:
def geocode(address,city=None,batch=True,sig=None)->dict:#将bath=True可以获取十个以下的地理编码
    """获取地理编码"""
    url = 'https://restapi.amap.com/v3/geocode/geo?parameters'
    params={
        'key': key_peng,
        'address':address,
        'city':city,
        'batch':batch,#batch 参数设置为 true 时进行批量查询操作
        'sig':sig,
        'output':'json'
    }
    response = requests.get(url,params=params)
    data = response.json()
    return data
#中大南方 = geocode(address='广东省广州市中山大学南方学院')
#print(中大南方)
#df_中大南方地理编码 = pd.json_normalize(中大南方['geocodes'])
#display(df_中大南方地理编码)
#中大南方地理编码 = 中大南方['geocodes'][0]['location']
#print("中大南方地理编码:",中大南方地理编码)


湖南省长沙市爱晚亭 = geocode(address='湖南省长沙市爱晚亭')
print(湖南省长沙市爱晚亭)
湖南省长沙市岳麓书院 = geocode(address='湖南省长沙市岳麓书院')
print(湖南省长沙市岳麓书院)

{'status': '1', 'info': 'OK', 'infocode': '10000', 'count': '1', 'geocodes': [{'formatted_address': '湖南省长沙市岳麓区爱晚亭', 'country': '中国', 'province': '湖南省', 'citycode': '0731', 'city': '长沙市', 'district': '岳麓区', 'township': [], 'neighborhood': {'name': [], 'type': []}, 'building': {'name': [], 'type': []}, 'adcode': '430104', 'street': [], 'number': [], 'location': '112.938016,28.180382', 'level': '兴趣点'}]}
{'status': '1', 'info': 'OK', 'infocode': '10000', 'count': '1', 'geocodes': [{'formatted_address': '湖南省长沙市岳麓区岳麓书院', 'country': '中国', 'province': '湖南省', 'citycode': '0731', 'city': '长沙市', 'district': '岳麓区', 'township': [], 'neighborhood': {'name': [], 'type': []}, 'building': {'name': [], 'type': []}, 'adcode': '430104', 'street': [], 'number': [], 'location': '112.955295,28.193112', 'level': '兴趣点'}]}


In [10]:
# B-2 准备walking 参数

爱晚亭_location = 湖南省长沙市爱晚亭['geocodes'][0]['location']
岳麓书院_location = 湖南省长沙市岳麓书院['geocodes'][0]['location']
print("(起点)爱晚亭location:",爱晚亭_location,"(终点)岳麓书院_location",岳麓书院_location)


(起点)爱晚亭location: 112.938016,28.180382 (终点)岳麓书院_location 112.955295,28.193112


In [13]:
爱晚亭_岳麓书院 = walking(爱晚亭_location,岳麓书院_location)
df_步行路径规划 = pd.json_normalize(爱晚亭_岳麓书院["route"]["paths"][0]['steps'])
display(df_步行路径规划)
df_步行路径规划["instruction"]

,instruction,orientation,road,distance,duration,polyline,action,assistant_action,walk_type
0,步行116米左转,[],[],116,93,"112.937986,28.180417;112.938147,28.180503;112....",左转,[],0
1,沿登高路向西北步行29米右转,西北,登高路,29,23,"112.939115,28.180486;112.938997,28.180694;112....",右转,[],0
2,步行242米右转,[],[],242,194,"112.938971,28.180734;112.9399,28.180734;112.94...",右转,[],0
3,向东步行117米向右前方行走,东,[],117,94,"112.940755,28.181706;112.940868,28.181784;112....",向右前方行走,[],0
4,向南步行40米向右前方行走,南,[],40,32,"112.941823,28.181766;112.941853,28.181471;112....",向右前方行走,[],0
5,向南步行101米左转,南,[],101,81,"112.941845,28.181402;112.941615,28.181128;112....",左转,[],0
6,向东步行467米左转,东,[],467,374,"112.942062,28.18079;112.942096,28.180825;112.9...",左转,[],0
7,沿麓山路向东北步行349米右转,东北,麓山路,349,279,"112.946797,28.181198;112.946797,28.181515;112....",右转,[],0
8,沿桃子湖路向东步行482米左转,东,桃子湖路,482,386,"112.948242,28.18401;112.949583,28.184071;112.9...",左转,[],0
9,沿学堂坡路向北步行566米直行,北,学堂坡路,566,453,"112.953099,28.183433;112.953108,28.183481;112....",直行,[],0


0              步行116米左转
1        沿登高路向西北步行29米右转
2              步行242米右转
3        向东步行117米向右前方行走
4         向南步行40米向右前方行走
5            向南步行101米左转
6            向东步行467米左转
7       沿麓山路向东北步行349米右转
8       沿桃子湖路向东步行482米左转
9       沿学堂坡路向北步行566米直行
10    沿爱民路向北步行512米到达目的地
Name: instruction, dtype: object

## 公车路径规划

In [14]:
def integrated(origin,destination,city,cityd=None,extensions='all',strategy=0,nightflag=1,date=None,time=None,sig=None)->dict:   
    url = 'https://restapi.amap.com/v3/direction/transit/integrated?parameters'
    params={
        'key':key_peng,
        'origin':origin,
        'destination':destination,
        'city':city,
        'cityd':cityd,
        'extensions':extensions,
        'strategy':strategy,
        'nightflag':nightflag,
        'date':date,
        'output':'json'
    }
    response = requests.get(url,params=params)
    data = response.json()
    return data



橘子洲头 = geocode('湖南省长沙市橘子洲头')
橘子洲头_location = 橘子洲头['geocodes'][0]['location']
print("(起点)岳麓书院_location:",岳麓书院_location,"(终点)橘子洲头_location:",橘子洲头_location)

(起点)岳麓书院_location: 112.955295,28.193112 (终点)橘子洲头_location: 112.961777,28.193799


In [16]:
岳麓书院_橘子洲头 = integrated(岳麓书院_location,橘子洲头_location,city='长沙市',extensions='all',strategy=0,nightflag=1)
df_bus = pd.json_normalize(岳麓书院_橘子洲头)
df_bus

,status,info,infocode,count,route.origin,route.destination,route.distance,route.taxi_cost,route.transits
0,1,OK,10000,1,"112.955295,28.193112","112.961777,28.193799",676,8.0,"[{'cost': '2.0', 'duration': '1326', 'nightfla..."


In [44]:

df_bus_岳麓书院_橘子洲头 = pd.json_normalize(岳麓书院_橘子洲头["route"]['transits'][0]['segments'][0]['bus']["buslines"])
df_bus_岳麓书院_橘子洲头


,name,id,type,distance,duration,polyline,bustimetag,start_time,end_time,via_num,via_stops,departure_stop.name,departure_stop.id,departure_stop.location,arrival_stop.name,arrival_stop.id,arrival_stop.location
0,地铁2号线(梅溪湖西--光达),430100020927,地铁线路,1137,294,"112.951436,28.197614;112.951978,28.197072;112....",0,0630,2300,0,[],溁湾镇,430100020927005,"112.951436,28.197614",橘子洲,430100020927006,"112.962520,28.195601"


In [45]:
df_bus_岳麓书院_橘子洲头['name']

0    地铁2号线(梅溪湖西--光达)
Name: name, dtype: object

# 关于百度通用物体和场景识别API调用

In [28]:
   '''
import requests
import base64

'''
通用物体和场景识别
'''

request_url = "https://aip.baidubce.com/rest/2.0/image-classify/v2/advanced_general"
# 二进制方式打开图片文件
f = open('G:\\API\\岳麓书院01.jpg', 'rb')
img = base64.b64encode(f.read())
baike_num = base64.b64encode(f.read())
params = {"image":img,"baike_num":baike_num}
access_token = '[24.88144b66f0e5c48842744b2e9959cfe3.2592000.1597557494.282335-21437406]'
request_url = request_url + "?access_token=" + access_token
headers = {'content-type': 'application/x-www-form-urlencoded'}
response = requests.post(request_url, data=params, headers=headers)
if response:
    print (response.json())
    '''    

{'log_id': 6039405127180011025, 'result_num': 5, 'result': [{'score': 0.888021, 'root': '', 'keyword': '岳麓书院'}, {'score': 0.690529, 'root': '建筑-传统建筑', 'keyword': '寺塔'}, {'score': 0.505357, 'root': '建筑-传统建筑', 'keyword': '中式传统建筑'}, {'score': 0.327303, 'root': '建筑-现代建筑', 'keyword': '沿街店面'}, {'score': 0.133775, 'root': '建筑-传统建筑', 'keyword': '古建筑'}]}


In [21]:
# encoding:utf-8
import requests 

# client_id 为官网获取的AK， client_secret 为官网获取的SK
host = 'https://aip.baidubce.com/oauth/2.0/token?grant_type=client_credentials&client_id=7H0ALlGvd9kfLQVtqQwX6isc&client_secret=Lp0iIIa8nMk4pDPqHcZLOKB981CuukIR'
response = requests.get(host)
if response:
    print(response.json())

{'refresh_token': '25.2e31fc867ee715825cde1d1297a7fafa.315360000.1910325494.282335-21437406', 'expires_in': 2592000, 'session_key': '9mzdA5KIONIgLywK1nj7+K8EvhbPpZMzipKzMmbMuEu/UcNq1yHXYnblKOGi0z7KZsZ/uNpPzA0TY/ok2oEvQgWwa6nuzg==', 'access_token': '24.88144b66f0e5c48842744b2e9959cfe3.2592000.1597557494.282335-21437406', 'scope': 'public vis-classify_dishes vis-classify_car brain_all_scope vis-classify_animal vis-classify_plant brain_object_detect brain_realtime_logo brain_dish_detect brain_car_detect brain_animal_classify brain_plant_classify brain_ingredient brain_advanced_general_classify brain_custom_dish brain_poi_recognize brain_vehicle_detect brain_redwine brain_currency brain_vehicle_damage wise_adapt lebo_resource_base lightservice_public hetu_basic lightcms_map_poi kaidian_kaidian ApsMisTest_Test权限 vis-classify_flower lpq_开放 cop_helloScope ApsMis_fangdi_permission smartapp_snsapi_base iop_autocar oauth_tp_app smartapp_smart_game_openapi oauth_sessionkey smartapp_swanid_verify 

In [34]:
import urllib
import base64
import json
import requests

#filename:图片名（本地存储包括路径）,plantnum展示的数量
def general(filename,plantnum):
    request_url = "https://aip.baidubce.com/rest/2.0/image-classify/v2/advanced_general"
    
    # 二进制方式打开图片文件
    f = open('G:\\API\\岳麓书院01.jpg', 'rb')
    img = base64.b64encode(f.read())
    
    params = dict()
    params['image'] = img
    params['baike_num'] = plantnum
    params = urllib.parse.urlencode(params).encode("utf-8")
    #params = json.dumps(params).encode('utf-8')
    
    
    request_url = request_url + "?access_token=" + access_token
    request = urllib.request.Request(url=request_url, data=params)
    request.add_header('Content-Type', 'application/x-www-form-urlencoded')
    response = urllib.request.urlopen(request)
    content = response.read()
    if content:
        #print(content)
        content=content.decode('utf-8')
        #print(content)
        data = json.loads(content)
        result=data['result']
        
        nums=min(plantnum,len(result))
        
        for i in range(0,nums):
            item=result[i]
            print ('名称:',item['keyword'])
            print ('可能性:',item['score'])
            baike_info=item['baike_info']
            print ('百科描述:',baike_info['description'])
            print ('百科链接:',baike_info['baike_url'])
            print ('百科图片:',baike_info['image_url'])
        
    
general('G:\\API\\岳麓书院01.jpg',1)

名称: 岳麓书院
可能性: 0.888021
百科描述: 岳麓书院是中国历史上赫赫闻名的四大书院之一，坐落于中国历史文化名城湖南长沙湘江西岸的岳麓山脚下，作为世界上最古老的学府之一，其古代传统的书院建筑至今被完整保存，每一组院落、每一块石碑、每一枚砖瓦、每一支风荷，都闪烁着时光淬炼的人文精神。1988年，岳麓书院建筑群被国务院批准为第三批全国重点文物保护单位。岳麓书院历经千年而弦歌不绝，学脉延绵。北宋开宝九年(公元976年)，潭州太守朱洞在僧人办学的基础上，由官府捐资兴建，正式创立岳麓书院。北宋祥符八年(公元1015年)，宋真宗召见岳麓山长周式，御笔赐书“岳麓书院”四字门额。嗣后，历经南宋、元、明、清各代，至清末光绪廿九年(公元1903年)，岳麓书院与湖南省城大学堂合并改制为湖南高等学堂，沿用书院旧址。中华民国15年(公元1926年)，湖南高等学堂正式定名湖南大学，仍就书院基址扩建至今。1986年湖南大学宣布完成修复岳麓书院，并正式对外开放。2005年湖南大学正式恢复岳麓书院，下设中国哲学研究所、历史研究所、中国思想文化研究所、中国书院研究中心和中国软实力文化研究中心等多个部门。2009年岳麓书院学科从本科、硕士研究生、博士研究生到博士后科研流动站的格局已经完备。正式成为为湖南大学的实体办学机构和科研基地，千年学脉再次延绵，千年学府再添光辉。今天的岳麓书院不仅是湖南大学的文史哲人才培养和研究基地，湖南省旅游胜地，更是是整个长沙市的文化窗口和文化名片。
百科链接: http://baike.baidu.com/item/%E5%B2%B3%E9%BA%93%E4%B9%A6%E9%99%A2/206944
百科图片: https://bkimg.cdn.bcebos.com/pic/fcfaaf51f3deb48fd3d7b926f61f3a292cf5788f


# 关于手写文本识别API调用

In [38]:
# encoding:utf-8

import requests
import base64

'''
手写文字识别
'''

request_url = "https://aip.baidubce.com/rest/2.0/ocr/v1/handwriting"
# 二进制方式打开图片文件
f = open('G:\\API\\文本识别01.jpg', 'rb')
img = base64.b64encode(f.read())

params = {"image":img}
access_token = '24.47e209f29e546b91fd84b47fa5c93265.2592000.1597559100.282335-21404324'
request_url = request_url + "?access_token=" + access_token
headers = {'content-type': 'application/x-www-form-urlencoded'}
response = requests.post(request_url, data=params, headers=headers)
if response:
    print (response.json())

{'log_id': 3942581198124530769, 'words_result_num': 4, 'words_result': [{'location': {'width': 2733, 'top': 15, 'left': 576, 'height': 661}, 'words': '窗间梅熟落蒂'}, {'location': {'width': 2792, 'top': 659, 'left': 578, 'height': 717}, 'words': '墙下笋成出林'}, {'location': {'width': 2981, 'top': 1179, 'left': 423, 'height': 1026}, 'words': '连雨不知春去'}, {'location': {'width': 2956, 'top': 1800, 'left': 524, 'height': 1145}, 'words': '-晴方觉夏深'}]}


In [35]:
# encoding:utf-8
import requests 

# client_id 为官网获取的AK， client_secret 为官网获取的SK
host = 'https://aip.baidubce.com/oauth/2.0/token?grant_type=client_credentials&client_id=IPBgIKvgpbrnhWm3U6Rk0i00&client_secret=2UbuVOn2dj7yC4jMZSDQSzh0OWGmkUZm'
response = requests.get(host)
if response:
    print(response.json())

{'refresh_token': '25.a1300bf7aae270db3c01883023fe4bc7.315360000.1910327100.282335-21404324', 'expires_in': 2592000, 'session_key': '9mzdDxNrfoIushGSQJtT39MLH9D2JvOOjBS+Be7G0fVDOmm5IEDhPSpTNM+PPneXuFIuRi35qcJp8qJfe2T2OM9meb9gvQ==', 'access_token': '24.47e209f29e546b91fd84b47fa5c93265.2592000.1597559100.282335-21404324', 'scope': 'public vis-ocr_ocr brain_ocr_scope brain_ocr_general brain_ocr_general_basic vis-ocr_business_license brain_ocr_webimage brain_all_scope brain_ocr_idcard brain_ocr_driving_license brain_ocr_vehicle_license vis-ocr_plate_number brain_solution brain_ocr_plate_number brain_ocr_accurate brain_ocr_accurate_basic brain_ocr_receipt brain_ocr_business_license brain_solution_iocr brain_qrcode brain_ocr_handwriting brain_ocr_passport brain_ocr_vat_invoice brain_numbers brain_ocr_business_card brain_ocr_train_ticket brain_ocr_taxi_receipt vis-ocr_household_register vis-ocr_vis-classify_birth_certificate vis-ocr_台湾通行证 vis-ocr_港澳通行证 vis-ocr_机动车购车发票识别 vis-ocr_机动车检验合格证识别 vis